# Measure Lifetime

format
Cs retention: [ NoRb, NoRberr, Rb, Rberr]
Rb retention: [ NoCs, NoCserr, Cs, Cserr]

In [73]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
from scipy import optimize, integrate
import os.path

def exp(t, a, tau):
    return a*np.exp(-t/tau)

def exp2(t, a, tau):
    return 1 - a*np.exp(-t/tau)

def loss_model(a0, a1, b, y):
    model = np.array([
        [0,a1,a0,b],
        [0,-a1,0,a0],
        [0,0,-a0,a1],
        [0,0,0,-(a0+a1+b)],
    ])
    return np.dot(model, y)

def loss_wrapper(a0, a1, b):
    return lambda t, y: loss_model(a0, a1, b, y)

def loss_model_hf(a0, a1, b3, b4, t1, y):
    r1 = 1./(2*t1)  # decay to other hf state
    model = np.array([
        [0,a1,a0,b3,b4],
        [0,-a1,0,a0,a0],
        [0,0,-a0,a1,a1],
        [0,0,0,-(a0+a1+b3+r1),r1],
        [0,0,0,r1,-(a0+a1+b4+r1)],
    ])
    return np.dot(model, y)

def loss_wrapper_hf(a0, a1, b3, b4, t1):
    return lambda t, y: loss_model_hf(a0, a1, b3, b4, t1, y)

In [23]:
itr_gap_ms = []
cs_data = []
rb_data = []
corr_ret_data = []
corr_ret_err = []

In [24]:
# gap = 8.5 ms
# dataset = '2018_07_19_20_35_41_csrb-collisions-31-hd'
itr_gap_ms.append(8.5)
cs_data.append([0.953987730061,0.0116037783345,0.91156462585,0.0234179067987])
rb_data.append([0.930555555556,0.0113233423732,0.91156462585,0.0234179067987])
corr_ret_data.append([[0.0136054421769,0.0748299319728],[0.0748299319728,0.836734693878]])
corr_ret_err.append([[0.00955483071186,0.0217015020725],[0.0217015020725,0.0304847174759]])

In [25]:
# gap = 30 ms
# dataset = '2018_07_19_18_34_50_csrb-collisions-41-hd'
itr_gap_ms.append(30)
cs_data.append([0.938906752412,0.0135808715829,0.924137931034,0.0219885574409])
rb_data.append([0.911627906977,0.013687752648,0.937931034483,0.0200372771632])
corr_ret_data.append([[0.0137931034483,0.0620689655172],[0.048275862069,0.875862068966]])
corr_ret_err.append([[0.00968569999804,0.0200372771632],[0.0178006786408,0.0273833205101]])

In [26]:
# gap = 60 ms
# dataset = '2018_07_19_18_34_50_csrb-collisions-41-hd'
itr_gap_ms.append(60)
cs_data.append([0.932467532468,0.0127891830676,0.912751677852,0.0231186104802])
rb_data.append([0.903125,0.0165350229036,0.845637583893,0.0295985058349])
corr_ret_data.append([[0.0268456375839,0.0604026845638],[0.127516778523,0.785234899329]])
corr_ret_err.append([[0.0132414210106,0.0195166767892],[0.0273255601904,0.0336425198568]])

In [27]:
# gap = 120 ms
# dataset = '2018_07_19_18_34_50_csrb-collisions-41-hd'
itr_gap_ms.append(120)
cs_data.append([0.943573667712,0.0129191361689,0.900662251656,0.0243416303385])
rb_data.append([0.91,0.0202361063448,0.880794701987,0.0263691816581])
corr_ret_data.append([[0.0264900662252,0.0728476821192],[0.0927152317881,0.807947019868]])
corr_ret_err.append([[0.0130684247688,0.0211492484686],[0.0236025475085,0.0320563073475]])

In [28]:
# gap = 300 ms
# dataset = '2018_07_19_18_34_50_csrb-collisions-41-hd'
itr_gap_ms.append(300)
cs_data.append([0.964412811388,0.0110516001996,0.93288590604,0.0204987791995])
rb_data.append([0.907172995781,0.0133288614474,0.892617449664,0.0253633379945])
corr_ret_data.append([[0.0201342281879,0.0469798657718],[0.0872483221477,0.845637583893]])
corr_ret_err.append([[0.0115068818169,0.0173345985998],[0.0231186104802,0.0295985058349]])

In [29]:
# gap = 600 ms
# dataset = '2018_07_19_18_34_50_csrb-collisions-41-hd'
itr_gap_ms.append(600)
cs_data.append([0.904255319149,0.0151743096289,0.843137254902,0.0294010831131])
rb_data.append([0.920177383592,0.012761757443,0.849673202614,0.0288933865574])
corr_ret_data.append([[0.0718954248366,0.0849673202614],[0.078431372549,0.764705882353]])
corr_ret_err.append([[0.0208835039387,0.0225423167423],[0.0217351695888,0.0342931228312]])

In [30]:
# gap = 1200 ms
# dataset = '2018_07_19_18_34_50_csrb-collisions-41-hd'
itr_gap_ms.append(1200)
cs_data.append([0.860788863109,0.0166742640558,0.748299319728,0.0357949040403])
rb_data.append([0.845291479821,0.0171235155465,0.843537414966,0.0299639289403])
corr_ret_data.append([[0.102040816327,0.149659863946],[0.0544217687075,0.69387755102]])
corr_ret_err.append([[0.0249664384425,0.0294232298746],[0.0187101121842,0.0380128420128]])

In [31]:
cs_data = np.array(cs_data)
cs_only = cs_data[:,0]
cs_only_err = cs_data[:,1]
cs_dual = cs_data[:,2]
cs_dual_err = cs_data[:,3]

rb_data = np.array(rb_data)
rb_only = rb_data[:,0]
rb_only_err = rb_data[:,1]
rb_dual = rb_data[:,2]
rb_dual_err = rb_data[:,3]

# Model System

In [32]:
corr_ret_data = np.array(corr_ret_data)
corr_ret_err = np.array(corr_ret_err)

# reformulate the model so it matches the expacted format for curve_fit
def flat_loss_model(t, a0, a1, b, p00, p01, p10):
    ts = t[0:len(t)/4]  # cut out the extra x values
    res = integrate.solve_ivp(
        loss_wrapper(a0, a1, b),
        (0,max(ts)),
        [p00, p01, p10, 1-p00-p01-p10],
        t_eval=ts
    )['y'].transpose()
    return res.flatten()


# do that simultaneous fit
popt, pcov = optimize.curve_fit(
    lambda t, b, p00, p01, p10: flat_loss_model(t, 1./11.3, 1./13.5, b, p00, p01, p10),
    np.tile(itr_gap_ms, 4)/1e3,
    corr_ret_data.flatten(),
    p0=[.5, corr_ret_data[0,0,0], corr_ret_data[0,0,1], corr_ret_data[0,1,0]],
    bounds=[(0,0,0,0), (np.inf, 1, 1, 1)],
    sigma=corr_ret_err.flatten()
)

# calculate p11
init = popt[1:4].tolist()
init.append(1-np.sum(init))

# numerically solve ode and plot
res = integrate.solve_ivp(
    loss_wrapper(1./11.3, 1./13.5, popt[0]),
    (0,10),
    init,
    t_eval=np.exp(np.linspace(np.log(0.001), np.log(3), 1000))
)
fig, ax = plt.subplots()
fmts = [['rx', 'yd'],['cs', 'ko']]
for i, y in enumerate(res['y']):
    ax.plot(res['t'], y, fmts[i/2][i%2][0]+'-', label=['None', 'Cs', 'Rb', 'Cs,Rb'][i])

# plot data
for i in range(2):
    for j in range(2):
        ax.errorbar(
                np.array(itr_gap_ms)/1e3, corr_ret_data[:,i,j],
                yerr=corr_ret_err[:,i,j],
                fmt=fmts[i][j]
            )
# ax.set_xscale("log", nonposx='clip')
ax.grid(which='both')
ax.set_xlabel('Interaction Time (s)')
ax.set_ylabel('Probability')
ax.legend(bbox_to_anchor=(0.25, 0.75), loc=1, borderaxespad=0.)
ax.set_xlim(0.00,1.5)
ax.set_ylim(0,1)

# save plot
pltfmts = ['pdf', 'png']
fn = 'conditional_lifetime_20180708.{}'
# for pfmt in pltfmts:
#     fig.savefig(fn.format(pfmt), dpi=300, format=pfmt)

FigureCanvasNbAgg()

In [33]:
popt

array([ 0.05937357,  0.01600044,  0.05795169,  0.06039106])

In [34]:
np.sqrt(np.diagonal(np.abs(pcov)))

array([ 0.02685073,  0.00666246,  0.00896018,  0.0092107 ])

## Finite Cs T1

In [35]:
# reformulate the model so it matches the expacted format for curve_fit
def flat_loss_model_hf(t, a0, a1, b3, b4, p00, p01, p10, t1):
    ts = t[0:len(t)/4]  # cut out the extra x values
    res = integrate.solve_ivp(
        loss_wrapper_hf(a0, a1, b3, b4, t1),
        (0,max(ts)),
        [p00, p01, p10, 1-p00-p01-p10, 0],
        t_eval=ts
    )['y'].transpose()
    res2 = res[:,:4]
    res2[:,-1] += res[:,-1]
    return res2.flatten()

In [52]:
# do that simultaneous fit
b4 = 0.85
t1 = 1.37
popt, pcov = optimize.curve_fit(
    lambda t, b3, p00, p01, p10: flat_loss_model_hf(t, 1./11.3, 1./13.5, b3, b4, p00, p01, p10, t1),
    np.tile(itr_gap_ms, 4)/1e3,
    corr_ret_data.flatten(),
    p0=[.1, corr_ret_data[0,0,0], corr_ret_data[0,0,1], corr_ret_data[0,1,0]],
    bounds=[(0,0,0,0), (np.inf, 1, 1, 1)],
    sigma=corr_ret_err.flatten()
)

# calculate p11
init = popt[1:4].tolist()
init.append(1-np.sum(init))
init.append(0)

# numerically solve ode and plot
res = integrate.solve_ivp(
    loss_wrapper_hf(1./11.3, 1./13.5, popt[0], b4, t1),
    (0,10),
    init,
    t_eval=np.exp(np.linspace(np.log(0.001), np.log(3), 1000))
)
print(res['y'].shape)
resy2 = res['y'][:-1]
resy2[-1] += res['y'][-1]
fig, ax = plt.subplots()
fmts = ['rx', 'yd', 'cs', 'ko']
for i, y in enumerate(resy2):
    ax.plot(res['t'], y, fmts[i][0]+'-', label=['None', 'Cs', 'Rb', 'Cs,Rb'][i])

# plot data
k=0
for i in range(2):
    for j in range(2):
        k += 1
        ax.errorbar(
                np.array(itr_gap_ms)/1e3, corr_ret_data[:,i,j],
                yerr=corr_ret_err[:,i,j],
                fmt=fmts[k-1]
            )
# ax.set_xscale("log", nonposx='clip')
ax.grid(which='both')
ax.set_xlabel('Interaction Time (s)')
ax.set_ylabel('Probability')
ax.legend(bbox_to_anchor=(0.25, 0.75), loc=1, borderaxespad=0.)
ax.set_xlim(0.008,3)
ax.set_ylim(0,1)

# save plot
pltfmts = ['pdf', 'png']
fn = 'conditional_lifetime_20180708.{}'
# for pfmt in pltfmts:
#     fig.savefig(fn.format(pfmt), dpi=300, format=pfmt)

(5, 1000)


FigureCanvasNbAgg()

In [37]:
popt

array([  4.22298583e-16,   1.63788784e-02,   5.75704471e-02,
         6.00086668e-02])

In [38]:
np.sqrt(np.diagonal(np.abs(pcov)))

array([ 0.03178765,  0.00704035,  0.00933724,  0.00959823])

# 42

In [40]:
itr_gap_ms42 = []
cs_data42 = []
rb_data42 = []
corr_ret_data42 = []
corr_ret_err42 = []

In [41]:
# gap = 8.5 ms
# dataset = '2018_07_19_13_02_40_csrb-collisions-42-high-density'
itr_gap_ms42.append(8.5)
cs_data42.append([0.970920840065,0.006753633904,0.958904109589,0.0164289708618])
rb_data42.append([0.919642857143,0.0114875610094,0.931506849315,0.0209045185838])
corr_ret_data42.append([[0.0,0.041095890411],[0.0684931506849,0.890410958904]])
corr_ret_err42.append([[0.0048348968731615734,0.0164289708618],[0.0209045185838,0.0258524902973]])

In [42]:
# gap = 30 ms
# dataset = '2018_07_19_13_09_30_csrb-collisions-42-high-density'
itr_gap_ms42.append(30)
cs_data42.append([0.964015151515,0.00810559327605,0.886524822695,0.0267107651473])
rb_data42.append([0.919417475728,0.00848121745782,0.851063829787,0.029982759516])
corr_ret_data42.append([[0.063829787234,0.0496453900709],[0.0851063829787,0.801418439716]])
corr_ret_err42.append([[0.0205863594403,0.0182924867954],[0.0234994019171,0.033596154394]])

In [43]:
# gap = 60 ms
# dataset = '2018_07_19_13_09_30_csrb-collisions-42-high-density'
itr_gap_ms42.append(60)
cs_data42.append([0.956656346749,0.0113302495161,0.880794701987,0.0263691816581])
rb_data42.append([0.936746987952,0.00944643016935,0.867549668874,0.0275857772297])
corr_ret_data42.append([[0.063829787234,0.0496453900709],[0.0851063829787,0.801418439716]])
corr_ret_err42.append([[0.0205863594403,0.0182924867954],[0.0234994019171,0.033596154394]])

In [44]:
# gap = 100 ms
# dataset = '2018_07_19_13_09_30_csrb-collisions-42-high-density'
itr_gap_ms42.append(100)
cs_data42.append([0.943014705882,0.00993896542765,0.82,0.0313687742827])
rb_data42.append([0.934920634921,0.00982740267647,0.8,0.0326598632371])
corr_ret_data42.append([[0.113333333333,0.0666666666667],[0.0866666666667,0.733333333333]])
corr_ret_err42.append([[0.025882927306,0.0203670030887],[0.0229718023608,0.0361068373539]])

In [45]:
# gap = 300 ms
# dataset = '2018_07_19_13_34_08_csrb-collisions-42-high-density-pt3'
itr_gap_ms42.append(300)
cs_data42.append([0.94434470377,0.00971383956275,0.787671232877,0.0338454648603])
rb_data42.append([0.900662251656,0.0140536468285,0.732876712329,0.0366180243116])
corr_ret_data42.append([[0.164383561644,0.0479452054795],[0.102739726027,0.684931506849]])
corr_ret_err42.append([[0.0306729998516,0.0176818269741],[0.025127656913,0.038445864606]])

In [46]:
# gap = 600 ms
# dataset = '2018_07_19_13_34_08_csrb-collisions-42-high-density-pt3'
itr_gap_ms42.append(600)
cs_data42.append([0.888519134775,0.0128379689766,0.598684210526,0.0397576065705])
rb_data42.append([0.848754448399,0.0151134696777,0.638157894737,0.0389764095422])
corr_ret_data42.append([[0.236842105263,0.164473684211],[0.125,0.473684210526]])
corr_ret_err42.append([[0.0344837797909,0.0300681344061],[0.0268248461025,0.040499145477]])

In [47]:
# gap = 1200 ms
# dataset = '2018_07_19_14_25_41_csrb-collisions-42-high-density-pt4'
itr_gap_ms42.append(1200)
cs_data42.append([0.856275303644,0.0157837042504,0.543046357616,0.04053834838])
rb_data42.append([0.84938704028,0.014968074378,0.543046357616,0.04053834838])
corr_ret_data42.append([[0.35761589404,0.0993377483444],[0.0993377483444,0.443708609272]])
corr_ret_err42.append([[0.0390047345037,0.0243416303385],[0.0243416303385,0.0404307337798]])

In [48]:
cs_data42 = np.array(cs_data42)
cs_only42 = cs_data42[:,0]
cs_only_err42 = cs_data42[:,1]
cs_dual42 = cs_data42[:,2]
cs_dual_err42 = cs_data42[:,3]

rb_data42 = np.array(rb_data42)
rb_only42 = rb_data42[:,0]
rb_only_err42 = rb_data42[:,1]
rb_dual42 = rb_data42[:,2]
rb_dual_err42 = rb_data42[:,3]

# Check that the loss is correlated

# Model System

In [54]:
corr_ret_data42 = np.array(corr_ret_data42)
corr_ret_err42 = np.array(corr_ret_err42)

# reformulate the model so it matches the expacted format for curve_fit
def flat_loss_model(t, a0, a1, b, p00, p01, p10):
    ts = t[0:len(t)/4]  # cut out the extra x values
    res = integrate.solve_ivp(
        loss_wrapper(a0, a1, b),
        (0,max(ts)),
        [p00, p01, p10, 1-p00-p01-p10],
        t_eval=ts
    )['y'].transpose()
    return res.flatten()


# do that simultaneous fit
popt42, pcov42 = optimize.curve_fit(
    lambda t, b, p00, p01, p10: flat_loss_model(t, 1./11.3, 1./13.5, b, p00, p01, p10),
    np.tile(itr_gap_ms42, 4)/1e3,
    corr_ret_data42.flatten(),
    p0=[.5, corr_ret_data42[0,0,0], corr_ret_data42[0,0,1], corr_ret_data42[0,1,0]],
    bounds=[(0,0,0,0), (np.inf, 1, 1, 1)],
    sigma=corr_ret_err42.flatten()
)

# calculate p11
init42 = popt42[1:4].tolist()
init42.append(1-np.sum(init42))

# numerically solve ode and plot
res42 = integrate.solve_ivp(
    loss_wrapper(1./11.3, 1./13.5, popt42[0]),
    (0,10),
    init42,
    t_eval=np.exp(np.linspace(np.log(0.001), np.log(3), 1000))
)

In [75]:
fig, ax = plt.subplots(ncols=2, figsize=(2*3.375,4), sharey=True)

fmts = [['rx', 'yd'],['cs', 'ko']]

##################
for i, y in enumerate(res42['y']):
    ax[0].plot(res42['t'], y, fmts[i/2][i%2][0]+'-')

# plot data
for i in range(2):
    for j in range(2):
        ax[0].errorbar(
                np.array(itr_gap_ms42)/1e3, corr_ret_data42[:,i,j],
                yerr=corr_ret_err42[:,i,j],
                fmt=fmts[i][j], label=[['None', 'Cs'], ['Rb', 'Cs,Rb']][i][j]
            )

###########
for i, y in enumerate(resy2):
    ax[1].plot(res['t'], y, fmts[i/2][i%2][0]+'-', label=['None', 'Cs', 'Rb', 'Cs,Rb'][i])

# plot data
k=0
for i in range(2):
    for j in range(2):
        k += 1
        ax[1].errorbar(
                np.array(itr_gap_ms)/1e3, corr_ret_data[:,i,j],
                yerr=corr_ret_err[:,i,j],
                fmt=fmts[i][j]
            )

#########
ax[0].legend(bbox_to_anchor=(0.75, 0.95), loc=1, borderaxespad=0.)
ax[0].set_ylabel('Probability', fontsize=14)
for r in range(2):
    # ax.set_xscale("log", nonposx='clip')
    ax[r].grid(which='both')
    ax[r].set_xlabel('Interaction Time (s)', fontsize=14)
    ax[r].set_xlim(0.00,1.5)
    ax[r].set_ylim(0,1)
    ax[r].text(1.3, 0.9, ['a)','b)'][r], fontsize=16)

plt.tight_layout()
# save plot
fmts = ['pdf', 'png', 'eps']
out = '/home/ebert/jupyter-notebooks/FNode/analyses/CollisionPaper/loss-fit/'
fn = os.path.join(out, 'loss_fit.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()